In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
import re

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

# special matplotlib argument for improved plots
from matplotlib import rcParams

In [2]:
# names of column headers
columns = ['Timestamp', 'IP Address', 'URL', 'User ID', 'City', 'Region/State', 'Country']

In [4]:
# load dataset
click_stream_data = pd.read_csv('./data/ClickStreamData.txt', sep='\t', usecols=columns, parse_dates=['Timestamp'])

In [5]:
click_stream_data.head()

,Timestamp,IP Address,URL,User ID,City,Region/State,Country
0,2012-03-22 01:17:00,99.122.210.17,https://www.ideatory-store.com/SH51443900/VD61...,6YBY0MC9-IMGB-F69S-8BLX-2NHAMVT8D0F3,Homestead,Florida,United States of America
1,2012-03-22 01:34:00,69.76.12.236,https://www.ideatory-store.com/SH51443900/VD30...,4MPY5H71-14AO-Z591-YCKB-K4288KV6V8WH,Coeur D Alene,Idaho,United States of America
2,2012-03-22 17:23:00,67.240.15.11,https://www.ideatory-store.com/SH51443900/VD96...,KTZML6WD-V8GA-JBW9-UWOS-XW2W0K38Y876,Queensbury,New York,United States of America
3,2012-03-22 17:05:00,67.240.15.11,https://www.ideatory-store.com/SH51443900/VD21...,KTZML6WD-V8GA-JBW9-UWOS-XW2W0K38Y876,Queensbury,New York,United States of America
4,2012-03-22 01:27:00,98.234.107.79,https://www.ideatory-store.com/SH51443900/VD05...,21V1Y4O9-B81P-4WRZ-TGQG-SPUCB9KUM326,Sunnyvale,California,United States of America


In [6]:
# headers of product category data
columns = ['URL', 'Category']

In [7]:
# load product data
product_data = pd.read_csv('./data/ProductCategoryData.txt', sep='\t', usecols=columns)

In [8]:
product_data.head()

,URL,Category
0,https://www.ideatory-store.com/,books
1,https://www.ideatory-store.com/SH51443900/VD21...,movies
2,https://www.ideatory-store.com/SH51443900/VD66...,games
3,https://www.ideatory-store.com/SH51443900/VD90...,electronics
4,https://www.ideatory-store.com/SH51443900/VD96...,computers


In [9]:
# headers of user data
columns = ['User ID', 'Number of friends', 'College Education']

In [10]:
# load user data
user_data = pd.read_csv('./data/UserProfileData.txt', sep='\t', usecols=columns)

In [11]:
user_data.head()

,User ID,Number of friends,College Education
0,72Y0WQ31-XQKU-4LTM-40ZX-PX37G6IK7S59,113,No
1,5GGQCY44-OK2D-XGS4-GJOY-U4RXBVXF7J3M,99,No
2,0SIFQG2V-TWFE-0RE7-8DL5-KYR2N4UMTYMY,0,No
3,WSCKFU9U-VQQM-99VL-ZZBS-2EGJ8RR9J55K,0,Yes
4,ZYV7RIS4-1G7X-ZCHF-ISUE-BF00R8NB8JHM,113,Yes


In [12]:
# process column names

def preprocess_column_names(col_name):
    col_name = col_name.lower()
    col_name = re.sub(r'[^a-z]', '', col_name)
    
    return col_name

In [13]:
click_stream_data.columns = click_stream_data.columns.map(preprocess_column_names)
user_data.columns = user_data.columns.map(preprocess_column_names)
product_data.columns = product_data.columns.map(preprocess_column_names)

## Exploratory Data Analysis

In [16]:
click_stream_data.columns

Index([u'timestamp', u'ipaddress', u'url', u'userid', u'city', u'regionstate', u'country'], dtype='object')

In [15]:
# Unique countries in the dataset
click_stream_data.country.unique()

array(['United States of America', 'Australia', 'Puerto Rico', 'Portugal',
       'Canada', 'Germany', 'South Africa', 'Japan', 'Philippines',
       'Colombia', 'Thailand', 'Italy', 'Northern Mariana Islands',
       'France', 'Switzerland', 'Jamaica', 'Czech Republic', 'Spain',
       'Guam', 'United Kingdom', 'Virgin Islands'], dtype=object)

In [18]:
# merge user and items data
user_merged = pd.merge(click_stream_data, user_data, on='userid')

In [21]:
merged_data = pd.merge(user_merged, product_data, on='url')

In [22]:
merged_data.head()

,timestamp,ipaddress,url,userid,city,regionstate,country,numberoffriends,collegeeducation,category
0,2012-03-22 01:34:00,69.76.12.236,https://www.ideatory-store.com/SH51443900/VD30...,4MPY5H71-14AO-Z591-YCKB-K4288KV6V8WH,Coeur D Alene,Idaho,United States of America,122,No,clothing
1,2012-03-21 22:46:00,69.76.12.236,https://www.ideatory-store.com/SH51443900/VD30...,4MPY5H71-14AO-Z591-YCKB-K4288KV6V8WH,Coeur D Alene,Idaho,United States of America,122,No,clothing
2,2012-03-21 22:47:00,69.76.12.236,https://www.ideatory-store.com/SH51443900/VD30...,4MPY5H71-14AO-Z591-YCKB-K4288KV6V8WH,Coeur D Alene,Idaho,United States of America,122,No,clothing
3,2012-03-21 22:57:00,69.76.12.236,https://www.ideatory-store.com/SH51443900/VD30...,4MPY5H71-14AO-Z591-YCKB-K4288KV6V8WH,Coeur D Alene,Idaho,United States of America,122,No,clothing
4,2012-03-22 01:27:00,69.76.12.236,https://www.ideatory-store.com/SH51443900/VD30...,4MPY5H71-14AO-Z591-YCKB-K4288KV6V8WH,Coeur D Alene,Idaho,United States of America,122,No,clothing


In [23]:
# group this data by userid
grouped_by_user = merged_data.groupby(['userid', 'category'])

In [28]:
transactiongrouped_by_user.size().unstack(1)

category,accessories,automotive,books,clothing,computers,electronics,games,handbags,home&garden,movies,outdoors,shoes,tools
userid,,,,,,,,,,,,,
0011GRB2-5JJ3-KJUO-RMCP-0TBHC08UAXLX,NaN,NaN,NaN,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002Y4OT6-5QCS-48PD-CDLK-0TSRUM620CLY,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
005O4L5Z-JUEX-BAPR-62BV-WGYFGA88C2TK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,NaN,35,NaN
0068W263-8Q86-BZER-U4K7-H07PX1BPFQ71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN
006EGSY4-FKS5-MT33-7TCF-RNHZL3EPJOXH,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00BNYZVF-XUKN-DIUE-YSR1-RNYMDFSJVBGR,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00JIXW46-IXC6-QOY9-E9AC-EORIJ8GG7CMJ,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00NOH5AW-PCQ1-Q3CX-IITO-RCNKQFKXV7XY,NaN,NaN,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN
00XPU8O6-F21Z-M5C6-T65C-6R4466IL7S07,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
